In [1]:
import os                                                                          
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"                                       
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import tensorflow as tf
session_config=tf.ConfigProto(
        gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.1, allow_growth=True), allow_soft_placement=True, log_device_placement=False)
sess = tf.Session(config=session_config)

from tensorflow.contrib.slim.nets import resnet_v2, resnet_utils
import tensorflow as tf
from tensorflow.contrib import layers as layers_lib
from tensorflow.python.ops import variable_scope
from tensorflow.contrib.layers.python.layers import utils
from tensorflow.contrib import slim
from tensorflow.nn import ctc_loss, conv2d
import numpy as np
resnet_v2_block = resnet_v2.resnet_v2_block
resnet_v2 = resnet_v2.resnet_v2

import matplotlib.pyplot as plt

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def resnet_v2_26_base(inputs,
                 num_classes=None,
                 is_training=True, # True - due to update batchnorm layers
                 global_pool=False,
                 output_stride=1, # effective stride 
                 reuse=None,
                 include_root_block=False, #first conv layer. Removed due to max pool supression. We need large receprive field
                 scope='resnet_v2_26'):
  
    """
    Tensorflow resnet_v2 use only bottleneck blocks (consist of 3 layers).
    Thus, this resnet layer model consist of 26 layers.
    I put stride = 2 on each block due to increase receptive field.

    """
    blocks = [
      resnet_v2_block('block1', base_depth=64, num_units=2, stride=2),
      resnet_v2_block('block2', base_depth=128, num_units=2, stride=2),
      resnet_v2_block('block3', base_depth=256, num_units=2, stride=2),
      resnet_v2_block('block4', base_depth=512, num_units=2, stride=2),
    ]
    return resnet_v2(
      inputs,
      blocks,
      num_classes,
      is_training,
      global_pool,
      output_stride,
      include_root_block,
      reuse=reuse,
      scope=scope)

def make_ocr_net(inputs, num_classes, is_training=True):
    '''
    Creates neural network graph.
    Image width halved and it's define timestamps width (feature sequence length) 
    No activation after output (no softmax), due to it's presence at ctc_loss() and beam_search().
    After resnet head features are resized to be [batch,1,width,channel], and after that goes 1x1 conv 
    to make anology of dense connaction for each timestamp.
    
    input: batch of images
    output: tensor of size [batch, time_stamps_width, num_classes]
    '''
    with tf.variable_scope('resnet_base', values=[inputs]) as sc:
        with slim.arg_scope([slim.conv2d],
                              activation_fn=None, normalizer_fn=None):
            net = resnet_utils.conv2d_same(inputs, 64, 7, stride=2, scope='conv1') #root conv for resnet
            #net = slim.max_pool2d(net, [3, 3], stride=2, scope='pool1') # due to enlarge of receptive field
            net = resnet_v2_26_base(net, output_stride=1, is_training = is_training)[0] # ouput is a tuple of last tensor and all tensors 
    with tf.variable_scope('class_head', values=[net]) as sc:
        net = tf.transpose(net, [0,3,1,2]) # next 4 lines due to column to channel reshape. [batch,c,h,w]
        _,c,h,_ = net.get_shape() # depth of input to conv op tensor should be static (defined)
        shape = tf.shape(net)
        net = tf.reshape(net, [shape[0], c*h, 1, shape[3]])
        net = tf.transpose(net,[0,2,3,1]) # back to [batch,h,w,c] = [batch,1,w,features*h]
        net = layers_lib.conv2d(net, num_classes, [1, 1], activation_fn=None) #CTC got softmax [batch,1,w,num_classes]
        net = tf.squeeze(net,1) #[batch,w,num_classes]
        return net

def get_prediction(output_net, seq_len, merge_repeated=False):
    '''
    predict by using beam search
    input: output_net - logits (without softmax) of net
           seq_len - length of predicted sequence 
    '''
    net = tf.transpose(output_net, [1, 0, 2]) #transpose to [time, batch, logits]
    decoded, prob = tf.nn.ctc_beam_search_decoder(net, seq_len, merge_repeated=merge_repeated)
    return decoded, prob


In [3]:
with open('../../data_generator/valid_charset.txt', 'r', encoding = 'utf-8') as f:
    valid_charset = f.read()
all_chars = list(valid_charset)
char_to_indx = dict(zip(all_chars,range(len(all_chars))))
num_classes = len(all_chars)

In [4]:
class OCRModel(object):
    def __init__(self, charset, input_image_batch, sequence_labels, is_training=True, learning_rate=1e-4,
                decay_steps=2**16, decay_rate=0.9, decay_staircase=False, momentum=0.9):
        self.charset = charset
        self.num_classes = len(charset)+2 #indexing starts from one, +empty char. Yes, I know that I got 0-th rudimental output, whatever
        self.is_training = is_training
        self.learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate 
        self.decay_staircase = decay_staircase
        self.momentum = momentum
        mapping_char = tf.constant(list(charset))
        self.table = tf.contrib.lookup.index_to_string_table_from_tensor(
                    mapping_char, default_value="blank")
        self.build_for_prediction(input_image_batch)
        
    def build_for_prediction(self, input_image_batch):
        
        self.input_image_batch = input_image_batch
        net = make_ocr_net(self.input_image_batch, self.num_classes, is_training=self.is_training)
        self.net = net
        self.prediction = get_prediction(net, [tf.shape(self.net)[1]], merge_repeated=False) # tuple(decoded, prob). decoded - list of top paths. I use top1
        pred_dense = tf.sparse_to_dense(self.prediction[0][0].indices, self.prediction[0][0].dense_shape, 
                                               self.prediction[0][0].values)
        self.pred_dense = pred_dense
        self.prediction_string = tf.reduce_join(self.table.lookup(tf.cast(pred_dense, tf.int64)-1))
        

In [5]:
import json
from skimage import io
from skimage.transform import rescale
from skimage.color import rgb2gray
def resize_height(image, max_dim):
    h, w = image.shape
    scale_factor = max_dim/h
    output_image = rescale(image.copy(), scale = scale_factor, order = 3)
    return output_image

In [6]:
graph = tf.Graph()
with graph.as_default():

    features = tf.placeholder(tf.float32, shape = [None, 32, None, 1])
    model = OCRModel(charset = all_chars, input_image_batch=features, 
                     sequence_labels=None, is_training=False)
    init = tf.global_variables_initializer()
    table_init = tf.tables_initializer()
    saver = tf.train.Saver()

In [7]:
from lev_dist import distance

In [8]:
LOG_DIR = 'log/'
with tf.Session(graph=graph) as sess:
#     run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
#     run_metadata = tf.RunMetadata()
    sess.run([init, table_init])
    try:
        saver.restore(sess, LOG_DIR + "model_lev_dist_0.3011674824197099.ckpt")
    except:
        print('cant restore')
    with open('../datasets/labeled_strings.json', 'r') as f:
        string_data = json.load(f)
    match = 0
    lev_dist = 0
    for example in string_data:
        image = io.imread('../datasets/' + example['path'])
        image = rgb2gray(image)
        image = resize_height(image, 32)
#         image = image/255 #as image scaled to [0,1]
        image = image - 0.5
        string = example['context']
        pred_string = sess.run([model.prediction_string], feed_dict={features: image[np.newaxis,...,np.newaxis]})
        pred_string = pred_string[0].decode('utf-8')
        print(pred_string.upper(), string)
        lev_dist += distance(pred_string.upper(), string) 
        if pred_string.upper() == string:
            match+=1

INFO:tensorflow:Restoring parameters from log/model_lev_dist_0.3011674824197099.ckpt


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


ДWHИA №WМ РИС ДЛИННОЗЕРНЫЙ 800Г ЯРМАРКА
DATR Н1=0=ФLТ Data : 11-08-2017
PYБ РУБ
89 89
78 78
1 КГ"=112,23PYБ 1 КГ = 112,23РУБ
PИСБЕЛЫИ ДЛИНQ3ЕPHЫИ 3 рис белый длинозерный 900г
MИСТРAЛЬ ОРИЁНТ МИСТРАЛЬ ОРИЕНТ
DMTU7~10~2O DATA :27-10-2017
4KГ =Я2 R9NVБ 1 кг = 82.89 руб
PYБ РУБ
60 60
74 74
РИС 3OЛЮTИСТЫИ 9Г AНГСTPEM РИС ЗОЛОТИСТЫЙ 900Г АНГСТРЕМ
DАTA :ЭO-О8~2O17 DATA :30-08-2017
 KГ=1NЛ 77NVБ 1 КГ =100.77 РУБ
PYC РУБ
69 69
90 90
РУБ РУБ
33 33
61 61
1 КГ"*48.90PУБ 1 КГ =45.90РУБ
PУБ РУБ
90 90
45 45
/C ДИJЕPНЫИ AАTСXИ РИС ДЛИННОЗЕРНЫЙ АЗИАТСКИЙ
ПACCИM ПАССИМ
ОATИ|Н0=10=H0 DATA : 30-10-2017
1 КГ=115,88PYБ 1 КГ =115,88РУБ
PУБ РУБ
94 94
57 57
1 КГ=8 90NVБ 1 КГ =63.90 РУБ
РYБ РУБ
90 90
63 63
1 KГ=77,8РY0 1 КГ =77,80РУБ
РУБ| РУБ
90 90
38 38
 КГ=18З,ЗJPYБ 1 КГ =153,33РУБ
РYБ| РУБ
33 33
61 61
1 KГ=77,80PУБ 1 КГ =77,80РУБ
PУБ РУБ
90 90
38 38
1 КГ=153,33PYБ 1 КГ =153,33РУБ
PYБ РУБ
33 33
61 61
СУПБОБОBЫИ 25Г ЯPMPKA СУП БОБОВЫЙ 250Г ЯРМАРКА
DATА :11-1О-2O17 DATA :11-10-2017
22 122
00) 00
РУБ РУБ
1_KГ=48

1Г=7Б.8DРYБ 1 КГ =75.80 РУБ
PYБ РУБ
90 90
3 37
1 KГ=1З0,34PУБ 1 КГ =130,34РУБ
PYБ| РУБ
17 1 7
65 65
1KГ=56 SOРУ5 1 КГ=56.90РУБ
PYБ РУБ
90 90
56 56
DAT :11-O-2O1 DATA :11-08-1017
1 KГ=12 23NVБ 1 КГ =112.23РУБ
РУБ РУБ
78 78
89 89
РИС ДЛЯ ПЛOВА 9O0ГР РИС ДЛЯ ПЛОВА 900ГР
DАTА :O4-O2-2O17 DATA :04-02-2017
1 КГ=91,29PУБ 1 КГ =91,29РУБ
PУБ РУБ
16 16
82 82
СУП БОБОBЫИ 25Г ЯPМAPKА СУП БОБОВЫЙ 250Г ЯРМАРКА
22 122
0 00
PYБ РУБ
 KГ=488,00PYБ 1 КГ =488,00РУБ
DATA :1L~1O~2O17 DATA :11-10-2017
РИС 5Х1OГР ДOHГУCTО МDD РИС 5X100 ДОН ГУСТО MDD
DАTA :19-O9-2D17 DATA :19-09-2017
1 KГ =75,80PУБ 1 КГ=75,80РУБ 
PУБ РУБ
90 90
37 37
РИС ДЛЯ ПЛОBА 900ГР РИС ДЛЯ ПЛОВА 900ГР
DATA_~02~2L DATA :04-02-2017
1 КГ=91,29РYБ 1 КГ =91,29РУБ
PYБ РУБ
6 1 6
82 82
PИС 5Х100ГР ДOН ГУСТО МDО РИС 5Х100ГР ДОН ГУСТО МDD
DЗTШ :19-O9~2Ф17 DATA :19-09-2017
1 KГ =7580РУБ 1 КГ =75,80РУБ
PYБ РУБ
90 90
37 37
РИС"ЭKCTPА"ЮУБAHСХИИ ЭЛИTНЫИ HBC "ЭКСТРА" КУБАНСКИЙ ЭЛИТНЫЙ
900Г 900Г
DАTA :O4-O8-2O17 DATA :04-08-2017
1 КГ=89,78PYБ 1 КГ =89.78РУ

DATА :O4-O0-2OL7 DATA :04-08-2017
К=T3Z3NУБ 1 КГ =89,78РУБ
PУC РУБ
80 80
80 80
РИC KУБАНСKИИ 9OГР РИС КУБАНСКИИ 900ГР
DАTА :20-LО-2O1 DATA :28-10-2017
1 КГ=79,77PУБ 1 КГ =79,77РУБ
РУБ РУБ
79 79
71 7 1
PIРIГIЫИДHПСT0 1NO РИС КРУГЛЫЙ ДОН ГУСТО 1КГ MDD
DATА :2O-1O-2O17 DATA :20-10-2017
1KГ =56,90PУБ 1 КГ =56,90РУБ
PУБ РУБ
90 90
56 56
ПACCИМ ПАССИМ
1 KГ=115,B8PYБ 1 КГ =115,88РУБ
PYБ РУБ
94 94
57 57
PИ №РУГПЫИПРИMЮRCXИ5Г РИС КРУГЛЫЙ ПРИМОРСКИЙ 500Г
ПАССИM ПАССИМ
DATA :22-IО-2OI7 DATA :22-10-2017
1KГ=115,88PУБ 1 КГ =115,88РУБ
PУБ РУБ
94 94
57 57
 КГ=56,90РYБ 1 КГ = 56,90РУБ
PУБ РУБ
90 90
56 56
РИС KPУГЛЫИ ПРИМОPСKИИ 5Г РИС КРУГЛЫЙ ПРИМОРСКИЙ 500Г
ПAССИМ ПАССИМ
DATA :22-LO~2O1 DATA :22-10-2017
1 КГ=115,88PУБ 1 КГ =115,88РУБ
PYБ РУБ
94 94
5 57
PИCДЛЯKAШИ8Г AГPOKУПЬTPA РИС ДЛЯ КАШИ 800Г АГРОКУЛЬТУРА
DАTА :O0-IО-2OI7 DATA :08-10-2017
1КГ=NЧ 1 КГ =40.16РУБ
PYБ РУБ
13 13
32 32
ФAСОЛЬ 8O0Г ШЕБЕКИНKАЯ ФАСОЛЬ 800Г ШЕБЕКИНСКАЯ
DАTА :26-O1-2O17 DATA :26-01-2017
1 КГ=122,6OPYБ 1 КГ =122,60РУБ
PYБ РУБ
08

...

PYБ. РУБ.
-17% -17%
РYБ. РУБ.
1OO ГP=6O.З1 100 ГР = 60.31
1KГ= 1519" 1 КГ =1519
P=61.76 100ГР = 61.76
РYБ. РУБ.
1KГ= 1499" 1 КГ = 1499
JO ГР= 259.98 100ГР = 259.98
OO Г 48.92 100ГР=48.92
РУБ. РУБ.
1OО ГP=58.33 100 ГР = 58.33
PУБ. РУБ.
)РYБ. РУБ.
1OOГРM 237" 100 ГР = 237 
1OO ГP= 27" 100 ГР = 237
-19% -19%
РYБ. РУБ.
PYБ. РУБ.
РYБ. РУБ.
1OO ГP=177" 100 ГР = 17777
-17% -17%
1KГ=399" 1 КГ = 399
PYБ. РУБ.
PУБ. РУБ.
1OO ГP=99" 100ГР = 99
"РУБ. РУБ.
1OOГP=94 100 ГР = 94
1OO ГP=78.78 100 ГР = 78.78
РYБ. РУБ.
10Q ГP= 224" 100 ГР = 224
РУБ. РУБ.
1KГ=2666" 1КГ = 2666
1ГP=0" 1 ГР = 0
OOГP=69.99 100 ГР = 69.99
PУБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
PУБ. РУБ.
РYБ. РУБ.
1KГ=259" 1КГ = 259
РYБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
K~В 1КГ = 1166
РYБ. РУБ.
РУБ. РУБ.
1KГ= 308" 1 КГ =308
1KГ=204" 1 КГ = 204
РУБ. РУБ.
100 ГP =99" 100 ГР = 99
PYБ. РУБ.
PУБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
РУБ. РУБ.
1 KГ= 2666" 1КГ = 2666
PYБ. РУБ.
PYБ. РУБ.
PYБ. РУБ.
PYБ. РУБ.
PYБ. РУБ.
PYБ. РУБ.
РУБ. РУБ.
10ORP-99" 100 

РYБ. РУБ.
PУБ. РУБ.
PYБ. РУБ.
1OOГP=99" 100 ГР = 99 
1OO ГP- 93" 100 ГР = 93
JS4 100 ГР = 184.54
РYБ. РУБ.
РYБ. РУБ.
1ОО ГРMZ6" 100 ГР = 76
1& Р=Z6" 100 ГР = 76
LOOГP=LOO.77 100 ГР = 100.77
1OO ГP=13O.17 100 ГР = 130.17
+1OO ГP=54.18 100 ГР = 54.18
РYБ. РУБ.
1OO ГР=52" 100 ГР = 52
1OO ГР-38.48 100 ГР = 38.48
1O0 ГP = L6.12 100 ГР= 16.12
PУБ. РУБ.
1OOГP= 239 100 ГР = 23965
1DO ГP=З6.6О 100 ГР = 36.80
1OO ГP= З5.34 100 ГР = 35.34
1OOГP=З5.З4 100 ГР = 35.34
L0O ГP=35* 100 ГР = 3557
РYБ. РУБ.
1OO ГP=89" 100ГР = 89
LOO RP=67" 100 ГР = 67
LOO ГP=L79" 100 ГР = 179 
10OГP=2766 100 ГР = 27.66
10O ГP= 55.98 100 ГР = 55.98
РYБ. РУБ.
1 KГ=36" 1 КГ = 3669
10ORP=53.31 100 ГР = 53.31
1OOГP=56.65 100 ГР = 56.65
РУБ. РУБ.
PYБ. РУБ.
РYБ. РУБ.
1Л=209* 1Л = 20995
РYБ. РУБ.
PYБ. РУБ.
1Л=2O9" 1Л = 209
1KГ = 157 1 КГ = 157
РУ6. РУБ.
РYБ. РУБ.
РYБ. РУБ.
 РУБ
РYБ. РУБ.
"РYБ. РУБ.
РYБ. РУБ.
PYБ. РУБ.
PУБ. РУБ.
РYБ. РУБ.
1KГ=339" 1 КГ = 339
РУБ. РУБ.
PYБ. РУБ.
РYБ. РУБ.
1KГ=259" 1 КГ = 259
PУБ. РУБ.
1KГ = 157" 1

1OO ГP=284" 100 ГР = 28495
РУБ. РУБ.
РYБ. РУБ.
1OORP=99.99 100 ГР = 99.99
1OO ГP=89" 100 ГР = 8993
1OOГP= L27.65 100 ГР = 127.65
1OOГP=78* 100 ГР = 7899
1OO ГP=87" 100 ГР = 8718
LOOГP=З7.98 100ГР = 37.98
1KГ=124* 1 КГ =12495
1OOГP=74" 100 ГР = 7442
1OO ГP=64.98 100 ГР = 64.98
LOOГP=15.79 100 ГР = 15.79
10O ГPL9.47 100 ГР = 19.47
1OOГP=2O" 100 ГР = 2097
РУБ. РУБ.
1OOP=L6.OO 100 ГР = 16.00
100ГР= 13" 100 ГР = 1302
LOОГP=З1 100 ГР = 3711
1OOГP=З1" 100 ГР = 3711
РУБ. РУБ.
PYБ. РУБ.
1OOГP=15" 100 ГР = 1594
PYБ. РУБ.
1OO ГP=89" 100 ГР = 8999
1OOГP=41* 100 ГР = 4120
1ODГP=41.55 100 ГР = 41.55
РYБ. РУБ.
1KГ=.222 1КГ = 22290
1OO ГP=59.O9 100 ГР = 59.09
10O ГP=З6.O0 100 ГР = 36.00
*PYБ. РУБ.
1OOГP=83.33 100 ГР = 83.33
J0O ШТ=29 100 ШТ = 2920
OOГP=37.99 100 ГР = 37.99
1OOГP=16" 100 ГР = 16
0ГР=27.66 100 ГР = 27.66
1OOГP=255.94 100 ГР = 255.94
1OOГP=45 100 ГР = 4500
PУБ. РУБ.
1OOГP=25.OO 100 ГР = 25.00
1OO ГP=29* 100 ГР = 2950
100 RP= 2З.ЗЗ 100 ГР = 23.33
1OO ГP=29" 100 ГР = 2950
РYБ. РУБ.
PYБ. РУ

1OOГP=245" 100 ГР = 24568
1OOГР= 106" 100 ГР = 106
10OГР= 155" 100 ГР = 155
РYБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
1KГ *1262" 1 КГ = 1262
РУБ. РУБ.
РУБ РУБ.
РY6. РУБ.
РУY6. РУБ.
PYБ. РУБ.
РYБ. РУБ.
10OГР= 125" 100 ГР = 12598
PYБ. РУБ.
1KГ= 1199" 1 КГ = 1199
1KГ=989* 1 КГ = 98990
100ГP= 114= 100 ГР = 11437
1 KГ = 1466" 1 КГ = 146660
PYБ. РУБ.
1OOГР=183 100 ГР = 18332
1 KГ=149 1 КГ = 14970
1 KГ = 1083 1 КГ = 1083
РYБ. РУБ.
100 ГР=66" 100 ГР = 6666
RР-83" 100 ГР = 83
РYБ. РУБ.
PYБ. РУБ.
110O ГР = 112" 100 ГР = 112
PYБ. РУБ.
РYБ. РУБ.
РYБ. РУБ.
10O ГP=5O" 100ГР = 5089
РYБ. РУБ.
РYБ. РУБ.
1KГ =241" 1 КГ = 241
1OOГP=21.6O 100 ГР = 21.60
РУБ. РУБ.
1OO RP=26" 100 ГР = 26
РYБ. РУБ.
1KГ=204* 1 КГ = 20495
РYБ. РУБ.
РYБ. РУБ.
1KГ=342" 1 КГ = 342
РYБ. РУБ.
РYБ. РУБ.
1KГ=242* 1 КГ = 24290
1KГ= 225" 1 КГ =22590
1ШТ=1899* 1 ШТ = 1899
РYБ. РУБ.
PУБ РУБ
РYБ. РУБ.
PУБ. РУБ.
1KГ=342* 1 КГ = 342
РYБ. РУБ.
1KГ=204* 1 КГ = 20495
1KГ= 127" 1 КГ = 127
1KГ=339* 1 КГ = 339
1KГ=339* 1 КГ =339
PYБ. РУБ.
1KГ=173" 1 КГ =1733

РYБ. РУБ.
РYБ. РУБ.
PYБ РУБ.
РYБ. РУБ.
PYБ. РУБ.
РYБ. РУБ.
K,ЦИЯ!/ АКЦИЯ!
1KГ=649~ 1 КГ = 64950
РYБ. РУБ.
PYБ. РУБ.
РYБ. РУБ.
PYБ РУБ
РYБ. РУБ.
РУБ РУБ.
РY6. РУБ.
1KГ= 3663" 1 КГ = 366333
1OO0 ГР= 1639" 1000 ГР = 1639
РYБ. РУБ.
1KГ=283* 1 КГ = 28330
1OОO RP = 1759" 1000 ГР = 1759
100 ГP =48 100 ГР = 4800
10O ГP = 66" 100 ГР = 66
10OГР=21" 100 ГР = 2110
PY6 РУБ.
PYБ. РУБ.
PYБ. РУБ.
1QО ГР= 37" 100 ГР = 37
РYБ. РУБ.
100 ГР=187 100 ГР = 187 
PY6. РУБ.
1И.ГP 1Б8 100 ГР = 158
РYБ. РУБ.
10О ГP= 99" 100 ГР = 99
РYБ. РУБ.
РУБ. РУБ.
РYБ. РУБ.
100 ГР=203 100 ГР = 203
АIЦИЯ! АКЦИЯ!
РYБ. РУБ.
1KГ= 1253~ 1 КГ = 125330
1OO ГР = 242" 100 ГР = 242
1KГ = 1499" 1 КГ = 1499
00 ГР-99" 100 ГР = 99
0ГР~203= 100 ГР = 203
1OORP=25" 100 ГР = 25
~ 1КГ = 1766
1KГ= 519" 1 КГ = 519
1KГ=399" 1 КГ = 39996
1KГ= 2266" 1 КГ =2266
PYБ. РУБ.
1KГ = 1166" 1 КГ = 1166
PYБ. РУБ.
РУБ. РУБ.
PYБ РУБ.
100 ГP=99" 100 ГР = 99
1KГ=679 1 КГ = 679
РYБ. РУБ.
РУБ. РУБ.
1 KГ = 1166" 1 КГ = 1166
1KГ=659* 1 КГ = 65999
РУБ. РУБ.
РYБ. РУБ.


In [9]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.31302064012848246
0.227830347458578


In [9]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.2986036127712828
0.25359988031566744


In [9]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.3310746717491847
0.22865317724501627


In [10]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.33543471234916805
0.22923289823091597


In [9]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.3545878951220188
0.18917604817294387


In [10]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.37406990236579035
0.16082582189475259


In [14]:
print(lev_dist/len(string_data))
print(match/len(string_data))

0.36911594371700374
0.15770280884168006
